In [ ]:
#### Get List of Every Result and Save ####
#### DO NOT DELETE ####

make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/"+ make + model +".html", 'w').close()
ids, urls = helpers.getlistings(make, model)
data = ''
listings = []
for i in range(0,len(ids)):
    urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=' + ids[i] + '&sort=td&page='
    urlend = '&results=items'
    pgend = 3
    tempurl = helpers.geturls(urlstart, urlend, pgend, urls)
    listings = tempurl

    def get_data(url):

        r = requests.get(url)

        soup = BeautifulSoup(r.content, "html.parser")
        return str(soup)


    souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in listings)


### WRITE DATA TO FILE ###
    delimiter = "%newline%"
    temp = delimiter.join(souplist)
    data = temp + delimiter + data
with open("./data/"+ make + model +".html", "a") as file:
    file.write(data)


display('done')

In [ ]:
carlist = []
Car = []
### READ DATA ###
with open("./data/"+ make + model +".html", "r") as file:
    test_data = file.read()
out = test_data.split(delimiter)

for i in range(1,len(listings)):
    soup = BeautifulSoup(out[i], 'html.parser')
    title = str(soup.find('title'))
    if "Parts" not in title and "Tool" not in title and "Memorabilia" not in title and "Tool Kit" not in title and "Removable" not in title and "Gearbox" not in title:
        loc = str(soup.findAll('a'))
        transmission = str(soup.findAll('li'))
        contents = str(soup.findAll('p'))
        essentials = soup.find('div', class_='essentials')

        #Get Car Description
        desc = helpers.getdesc(listings[i])


        #Get Year
        if '19' in title:
            index = title.index('19')
            year = title[index:index+4]
        else:
            year = 'N/A'

            
        #Check if Sold and get highest bid
        sold, price = helpers.getsaleprice(title, loc)


        #Get Transmission
        i = 0
        trans = 'Manual'
        if 'Automatic' in transmission:
            trans = 'Automatic'

        #Get Location

        town, state = helpers.getlocation(loc)

        
        #Check for Project
        if 'project' in title or 'Project' in title:
            project = 1
        else:
            project = 0
        
        #Get Lot Number
        index = title.index('Lot #')
        lotnumb = title[index+5:index+12].partition(')')[0]
        
        #Get Month
        month = helpers.getmonth(title)
        
        #Get Year
        index = title.index('(')
        yearsold = title[index-5:index-1]
        
        
        #Get Engine and Number Matching
        numbmatch = 0
        
        if 'Numbers-Matching' in transmission:
            numbmatch = 1

        engine = helpers.getengine(transmission)
        
        #Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        rust, refurbished, restored, scratch, paintbub, metalrepair, hardtop, overdrive = helpers.getindicators(contents)
            
            
        #Get Mileage
        miles, milestmu = helpers.getmileage(transmission)

        #Get Descriptions of options and chassis #

        temp = helpers.getenginedesc(essentials, engine)
        chassis = temp[0]
        specialdesc = temp[1]
        mileagedesc = temp[2]
        enginedesc = temp[3]
        transdesc = temp[4]
        paintdesc = temp[5]
        interiordesc = temp[6]
        carbdesc = temp[7]
        wheeldesc = temp[8]
        brakedesc = temp[9]
        engine = temp[10]

        #Check for bespoke bodies
        if 'Bertone' in title:
            bertone = 1
        else:
            bertone = 0
        if 'Zagato' in title:
            zagato = 1
        else:
            zagato = 0

        Car = [lotnumb, desc, trans, year, miles, milestmu, engine, numbmatch, hardtop, overdrive, scratch, paintbub, metalrepair, rust, refurbished, restored, project, sold, month, yearsold, price, town, state, zagato, bertone, chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc]
        carlist.append(Car)
    else:
        print('Discarded')
    # index = transmission.index('aint')
    # print(transmission[index-20:index+20])

df = pd.DataFrame(carlist, columns=['Lot Number', 'Description', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)','Engine', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', 'Rust','Refurbished','Restored', 'Project', 'Sold', 'Month', 'Year', 'Price', 'City', 'State', 'Zagato', 'Bertone', 'Chassis', 'Special Description', 'Mileage Description', 'Engine Description', 'Transmission Description','Paint Description', 'Interior Description','Carburetor Description', 'Wheels Description', 'Brakes Description'])

display(df)

